In [13]:
#%pip install spacy
#%pip install nltk

In [14]:
import pandas as pd
import numpy as np

In [15]:
# import csv
#df = pd.read_csv('combined_resume_data.csv')
#totalRows = df.shape[0]

In [16]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

#nltk.download('punkt')
#nltk.download('stopwords')

In [17]:
# tokenise and remove stopwords
#stop_words = set(stopwords.words('english'))
#resume_tokens = word_tokenize(resume_skills.lower())

# convert to set to remove duplicates then back to list
#unique_tokens = list(set(resume_tokens))

#filtered_tokens = [word for word in resume_tokens if word.isalpha() and word not in stop_words]

In [18]:
import spacy
from spacy.pipeline import EntityRuler
from collections import defaultdict
#spacy.cli.download("en_core_web_sm")

In [19]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

def remove_stopwords(text):
    """
    Remove stop words from the input text.
    
    Args:
        text (str): Input text string
        
    Returns:
        str: Text with stop words removed
    """
    # Download stopwords if not already downloaded
    try:
        nltk.data.find('corpora/stopwords')
    except LookupError:
        nltk.download('stopwords')
        
    # Download punkt tokenizer if not already downloaded
    try:
        nltk.data.find('tokenizers/punkt')
    except LookupError:
        nltk.download('punkt')
    
    # Get English stop words
    stop_words = set(stopwords.words('english'))
    
    # Tokenize text
    word_tokens = word_tokenize(text.lower())
    
    # Remove stop words
    filtered_words = [word for word in word_tokens if word not in stop_words and word.isalnum()]
    
    # Join the filtered words back into a string
    filtered_text = ' '.join(filtered_words)
    
    return filtered_text

In [20]:
resume = """
DAVID CHEN
Software Developer
Boston, MA | david.chen@email.com | (617) 555-1234 | linkedin.com/in/davidchen

SUMMARY
-------
Dedicated Software Developer with 4 years of experience in full-stack web development. Proficient in JavaScript, Python, and Java with expertise in React.js and Django frameworks. Passionate about creating efficient, scalable solutions and continuously learning new technologies.
Database management, Database administration, Database security, Database modelling, Stress testing, load testing

SKILLS
------
Languages: JavaScript, Python, Java, TypeScript, HTML5, CSS3, SQL, C++, C#. C
Frameworks & Libraries: React.js, Django, Spring Boot, Node.js, Express.js, jQuery, Bootstrap
Tools & Technologies: Git, Docker, AWS (EC2, S3), RESTful APIs, MongoDB, PostgreSQL, Redis, MS SQL SERVER, GRAPHSQL, NOSQL, FIREBASE, gitlab
Methodologies: Agile/Scrum, Test-Driven Development, CI/CD

PROFESSIONAL EXPERIENCE
----------------------
SOFTWARE DEVELOPER | Innovate Solutions | Boston, MA | 2022-Present
- Developed and maintained multiple web applications using React.js and Django, serving 20,000+ users
- Implemented authentication system using OAuth 2.0, improving security and user experience
- Containerized applications using Docker, reducing deployment time by 40%
- Collaborated with UX designers to implement responsive, accessible user interfaces
- Participated in code reviews and mentored junior developers

JUNIOR SOFTWARE DEVELOPER | TechStart Inc. | Cambridge, MA | 2020-2022
- Built RESTful APIs using Django and integrated with front-end React components
- Optimized database queries in PostgreSQL, improving application performance by 30%
- Created automated testing suites with Pytest and Jest, achieving 85% code coverage
- Assisted in migrating legacy PHP application to modern React/Django stack
- Participated in daily stand-ups and bi-weekly sprint planning meetings

EDUCATION
---------
BACHELOR OF SCIENCE IN COMPUTER SCIENCE | Boston University | 2020
- GPA: 3.7/4.0
- Relevant coursework: Data Structures, Algorithms, Database Systems, Web Development
- Senior Project: Developed a task management application using MERN stack

PROJECTS
--------
INVENTORY MANAGEMENT SYSTEM | github.com/davidchen/inventory-app
- Built a full-stack inventory management application using React, Node.js, and MongoDB
- Implemented barcode scanning functionality using WebRTC
- Deployed application to AWS with CI/CD pipeline using GitHub Actions

WEATHER VISUALIZATION DASHBOARD | github.com/davidchen/weather-viz
- Created an interactive dashboard using D3.js to visualize historical weather data
- Integrated with OpenWeatherMap API to fetch and display real-time weather information
- Implemented responsive design principles for mobile compatibility

CERTIFICATIONS
-------------
- AWS Certified Developer – Associate
- MongoDB Certified Developer
- Certified Scrum Master (CSM)

ADDITIONAL INFORMATION
---------------------
- Languages: English (Native), Mandarin Chinese (Fluent)
- Volunteer: Code instructor at local community center teaching Python to high school students
- Interests: Hiking, photography, contributing to open-source projects" \" \
"""

In [21]:
elliot_resume = """
Tan Jun-You Elliot
Elliot990@live.com | https://www.linkedin.com/in/tanelliot/

EDUCATION 
Bachelor of Science, Information Systems	2022 - Aug 2026 (Expected)
Singapore Management University
Diploma in Financial Informatics	2017-2020
Singapore Polytechnic
GPA 3.695 / 4.00
Participated in IMDA iPREP Programme (2019-2020)
Director’s Honour Roll for AY2019/2020 
 
WORK EXPERIENCE 	 
Singapore Management University - Teaching Assistant	Aug 2024 - Nov 2024
Assist in teaching students IS210: Business Process Analysis and Solutioning
The module teaches students to analyse and optimise business processes through modeling and technological solutions, preparing them to align enterprise objectives with effective IT solution architectures
KPMG Singapore - Technology Risk Intern	May 2024 - Nov 2024
Identify and evaluate technology risks across various domains in cybersecurity, data accuracy, privacy, and system reliability
Conduct IT audits to assess effectiveness of IT controls and recommend improvements
Singapore Management University - Teaching Assistant	Jan 2024 - Apr 2024
Assist students in their coding needs for IS113: Web Application Development I
The module teaches students program dynamic web applications using PHP, design dynamic web pages that present different contents reflecting contents stored in a database and allow users to modify database contents through the webpages
Zhilong Construction and Engineering Pte Ltd - Administrative Assistant	Jun 2023 - Aug 2023
Organised and updated company’s inventory list with reference to suppliers’ catalogues
Utilised Excel for data entry including its functions and formulas for cleaning and organising
Created internal catalogues for the sales engineering team
United Overseas Bank - Business Technology Services Intern	Sept 2019 - Apr 2020
Expedited the aggregation, consolidation and feeding of market data into centralised database management system
Oversee deployment of new fixes and features into the stage and production environments.
Executed and documented SIT and UAT test cases
Improve team's communication efficiency by creating automated forms in Word and Excel with VBA

NOTABLE PROJECTS
Data Analysis & Machine Learning
Final Year Project (On-Going)
Developed automated data cleaning pipeline integrating OpenAI's GPT API with pandas/numpy
Created used car price prediction model with XGBoost and Rule-Based modelling
Created required backend APIs for frontend integration
SMU Business Intelligence & Analytics Datathon 2025
Analysed unstructured data from Internal Security Department
Developed NLP-powered analytics tool using NER and Topic Modeling
NUS Rightship Hackathon 2024
Analysed vessel emissions data at Singapore ports
Conducted impact analysis of electrification on emissions
National Data Science Challenge 2019
Built ML model using Keras and TensorFlow
Achieved 75% accuracy in product information extraction
Ranked 83rd on competition leaderboard
Web Application Development
Automated Cleaning Assignment Scheduler for Abba Maintenance Services
Developed performance monitoring system integrating customer, workforce, and operational metrics
Built interactive visualisations with dynamic filtering and cross-dashboard interactions
Created required backend APIs for frontend integration


KEY SKILLS

Python, RStudio
SQL Database Management
Web Scraping
Tableau, Power BI
Excel, PivotTable, VBA



AWS, EC2, RDS
Data Analysis, Visualisation
Machine Learning Modelling
Dashboard Design
Process Optimisation


Cross-team Collaboration
Analytical Thinking
Attention to Detail
Stakeholder Management
Project Management




CERTIFICATIONS
Java Certified Foundations Associate
Issued by Oracle
Google IT Automation with Python Professional Certificate
Issued by Coursera
Google Data Analytics Professional Certificate
Issued by Coursera
MTA: Introduction to Programming Using Python
Issued by Microsoft

CO-CURRICULAR ACTIVITIES
Singapore Management University Athletics Club (Competitive) - Member	Aug 2022 - Dec 2023
Long distance runner
Represented school in the Singapore Athletics Interclub Games in 2022
"""

In [22]:
job_posting = """
Senior Software Engineer - Machine Learning
Location: San Francisco, CA (Hybrid)
Salary Range: $150,000 - $180,000 annually
Job Type: Full-time
About Us
TechInnovate AI is a fast-growing technology company dedicated to building intelligent solutions that transform how businesses operate. Our AI-powered platform helps companies automate processes, gain insights from their data, and make better decisions. Join our team of passionate technologists as we push the boundaries of what's possible with machine learning and artificial intelligence.
Job Description
We are seeking an experienced Senior Software Engineer with machine learning expertise to join our AI Products team. In this role, you will design, build, and maintain scalable machine learning systems that power our core products. You'll collaborate with data scientists, product managers, and other engineers to implement algorithms and create robust ML infrastructure.
Responsibilities

Design and implement scalable, production-grade machine learning systems
Write clean, maintainable, and well-tested code in Python and Java
Collaborate with data scientists to deploy ML models to production environments
Optimize ML pipelines for performance, reliability, and accuracy
Work with distributed systems and cloud infrastructure (AWS)
Mentor junior engineers and contribute to architectural decisions
Implement CI/CD pipelines for ML model deployment and monitoring
Participate in code reviews and technical design discussions

Requirements

Bachelor's degree in Computer Science, Engineering, or related field (Master's preferred)
5+ years of software engineering experience
3+ years of experience with machine learning frameworks (TensorFlow, PyTorch)
Strong programming skills in Python and experience with Java
Experience with SQL and NoSQL databases (PostgreSQL, MongoDB)
Proficiency with cloud platforms, preferably AWS (EC2, S3, Lambda)
Knowledge of containerization and orchestration (Docker, Kubernetes)
Understanding of software design patterns and best practices
Experience with microservices architecture and RESTful APIs
Strong problem-solving and debugging skills

Preferred Qualifications

Experience with natural language processing (NLP) or computer vision
Knowledge of real-time prediction systems
Familiarity with data streaming technologies (Kafka, Spark)
Experience with MLOps tools and practices
Contributions to open-source projects
Experience with agile development methodologies

Benefits

Competitive salary and equity package
Comprehensive health, dental, and vision insurance
401(k) plan with employer matching
Flexible work arrangements (hybrid/remote options)
Professional development stipend
Unlimited PTO policy
Wellness programs and gym membership
Company-sponsored team events and offsites
Catered lunches and stocked kitchen

TechInnovate AI is an equal opportunity employer. We celebrate diversity and are committed to creating an inclusive environment for all employees.
To Apply: Please submit your resume and a brief cover letter describing your interest in the position and relevant experience to careers@techinnovateai.com with the subject line "Senior Software Engineer - ML".
"""

In [23]:
# regex patterns

import re
import random
from collections import defaultdict

# regex patterns for better matching
patterns = {
    "Programming Language (PL)": r"\b(go|c\+\+|c#|c|java(?:script)?|python|php|r|ruby|swift|assembly|matlab|pascal|perl|rust|v(?:ba|b|\.net)|kotlin|objective-c)\b",
    
    "Framework (FW)": r"\b(?:django|flask|spring(?:\sframework|\sboot)?|react(?:.js|native)?|\w+\.js|angular|vue|express|fastapi|asp\.net|\.net|laravel|ruby\son\srails|symfony|meteor|gatsby|svelte|phoenix|cake|code)\b",
    
    "Database (DB)": r"\b((?:\w+)ql(?:\w+)?|(?:\w+)db|\w+store|\w+\sstorage|\w+base(?:\s\w+){0,1}|data(?:\s(?!analysis|analytics|science|visualisation|visualization)\w+)|redis|cassandra)\b",
    
    "Cloud Platform (CP)": r"\b(cloud(?:\s\w+)?|aws(?:\s+\w+){0,3}|azure(?:\s+\w+){0,3}|gcp|google\s+cloud(?:\s+\w+){0,3}|amazon\s+web\s+services(?:\s+\w+){0,3}|(?:microsoft\s+)?azure(?:\s+\w+){0,3}|ibm\s+cloud(?:\s+\w+){0,3}|oracle\s+cloud(?:\s+\w+){0,3}|digital\s*ocean|heroku|serverless|iaas|paas|saas|faas|baas|caas|s3\s+bucket|ec2(?:\s+instance)?|lambda|elastic\s+bean\s*stalk|ecs|eks|fargate|sqs|sns|rds|redshift)\b",
    
    "DevOps (DO)": r"\b(devops|devsecops|ci(?:/cd)?|continuous (?:integration|delivery|deployment)(?:/continuous (?:delivery|deployment))?|version\scontrol|git(?:hub|lab)?|\w+ci|jenkins|configuration\smanagement|ansible|puppet|terraform|container(?:ization|isation)?|docker(?:ization|ized|isation|ised)?(?:\sswarm)?|kubernetes|helm|monitor(?:ing)|prometheus|grafana|\w+\stest(?:ing)?|selenium|junit|cypress|jest|load\sbalancing|scal(?:e|ing|ability)|disaster\srecovery|chaos\sengineering)\b",
    
    "Network & Security": r"\b(network(?:\s\w+)?|security(?:\s\w+)?|routing|switching|firewall(?:\s\w+)?|vpn|tcp(?:/ip)?|dns|dhcp|ccna(?:/ccnp)?|sd-wan|vlans|subnet|risk|incident\s\w+|identity\sand\saccess\smanagement|iam(?:s)?|mfa|multi-factor\sauthentication|vulnerability\s\w+|crowdstrike|carbon\sblack|cisco|defense|sso|oauth|saml|hsm|\w+\sencryption|owasp\szap|\w+\shacking)\b",

    "Data Analysis & Science": r"\b(?:pandas|numpy|scikit-learn|tensorflow|data\s(?:analytics|analysis|science)|power\s*bi|excel|tableau|matplotlib|data\s*visualisation|data\s*visualization|visualizing\s*data|AI|artificial\s*intelligence|machine\s*learning|ML|NLP|natural\s*language\s*processing|text\s*analytics|language\s*model(?:s|ing)?|transaction\s*management|data\s*transaction(?:s)?)\b",
    
    "Software Engineering (SWE)": r"\b(?:software\s*development|design\s*patterns|unit\s*testing|full\s*stack|fullstack|full-stack|software\s*engineer(?:ing)?|junior\s*(?:software\s*)?engineer|senior\s*(?:software\s*)?engineer|staff\s*engineer|principal\s*engineer|software\s*engineer\s*intern(?:ship)?|code\s*optimi(?:s|z)ation|performance\s*tuning|code\s*refactoring|refactor(?:ing)?|microservice(?:s)?|MSA|microservice\s*architecture|containerization|docker|kubernetes|k8s|code\s*review(?:s)?|peer\s*review(?:s)?|eclipse|eclipse\s*ide|hibernate|hibernate\s*orm|jquery|rest(?:ful)?\s*api(?:s)?|rest\s*api(?:s)?|restful\s*web\s*service(?:s)?|api\s*development|web\s*service(?:s)?|object\s*oriented\s*programming|oop|object\s*oriented\s*design|jsp|java\s*server\s*pages|rpc|remote\s*procedure\s*call|j2ee|java\s*ee|jvm|java\s*virtual\s*machine|jax|jax-rs|jax-ws|apache|apache\s*(?:kafka|tomcat|maven|ant|struts|camel|spark|hadoop|flink)|bootstrap|front-end\s*framework|rabbitmq|message\s*(?:queue|broker)|front[\s-]end|back[\s-]end|web\s*design|ui|user\s*interface|ux|user\s*experience|ui\/ux|front[\s-]end\s*development|back[\s-]end\s*development|web\s*development)\b",
    
    "Project Management (PM)": r"\b(?:agile|jira|trello|asana|kanban|prince2|stakeholder\s*management|(?<!certified\s)scrum(?!\smaster))\b",

    "Education Certification (EC)": r"\bcertified\s+scrum\s+master\b|\bcsm\b|\bpmp\b|\baws\s+certified\b|\bazure\s+certified\b|\bgcp\s+certified\b|\bcissp\b|\bccna\b|\bceh\b|\bcomptia\b|\bcisa\b|\bcism\b|\b(?:bachelor(?:\s+of\s+(?:science|engineering|computer\s+science|information\s+technology|information\s+systems|cybersecurity|data\s+science|software\s+engineering))?(?:\s+in\s+(?:computer\s+science|computer\s+engineering|information\s+technology|information\s+systems|cybersecurity|data\s+science|software\s+engineering|artificial\s+intelligence|machine\s+learning))?|B\.?S\.?|B\.?E\.?|B\.?C\.?S\.?|B\.?Tech\.?|(?<!certified\s)master(?:\s+of\s+(?:science|engineering|computer\s+science|information\s+technology|information\s+systems|cybersecurity|data\s+science|software\s+engineering))?(?:\s+in\s+(?:computer\s+science|computer\s+engineering|information\s+technology|information\s+systems|cybersecurity|data\s+science|software\s+engineering|artificial\s+intelligence|machine\s+learning))?|M\.?S\.?|M\.?E\.?|M\.?C\.?S\.?|M\.?Tech\.?|phd|ph\.?d\.?|doctorate|doctor\s+of\s+philosophy(?:\s+in\s+(?:computer\s+science|computer\s+engineering|information\s+technology|data\s+science))?|specialisation\s+in\s+(?:software|data|cloud|security|networking|ai|machine\s+learning)|specialization\s+in\s+(?:software|data|cloud|security|networking|ai|machine\s+learning)|minor\s+in\s+(?:computer\s+science|information\s+technology|data\s+science|software\s+engineering)|major\s+in\s+(?:computer\s+science|information\s+technology|data\s+science|software\s+engineering))\b",
    
    "Soft Skills (SS)": r"\b(?:communication(?:\s+skills?)?|leadership(?:\s+skills?)?|teamwork|problem[\s-]solving|critical[\s-]thinking|adaptability|time[\s-]management|conflict[\s-]resolution|decision[\s-]making|emotional[\s-]intelligence|interpersonal(?:\s+skills?)?|presentation(?:\s+skills?)?|public[\s-]speaking|negotiation(?:\s+skills?)?|creativity|flexibility|collaboration|self[\s-]motivated|detail[\s-]oriented|work[\s-]ethic|customer[\s-]service|team[\s-]player|team[\s-]building|active[\s-]listening|project[\s-]coordination|mentoring|coaching|verbal(?:\s+communication)?|written(?:\s+communication)?|organizational(?:\s+skills?)?|multitasking|prioritization|stress[\s-]management|cultural[\s-]awareness|strategic[\s-]thinking)\b"

}

#test_text = df.iloc[random.randint(1, totalRows)]['clean_text']
#modified_text = remove_stopwords(test_text_2)
#modified_text = resume

extracted_entities = defaultdict(set)



In [24]:
# Test resume

modified_resume = resume
# Apply regex patterns with wildcard matching
for label, pattern in patterns.items():
    # Define the replacement function inside the loop
    # This creates a new function for each iteration
    def replacement_func(match_obj):
        match_text = match_obj.group(0)
        
        # Add the match to extracted entities
        if isinstance(match_text, tuple):  # Handle tuple case from capture groups
            extracted_entities[label].update(map(str.lower, match_text))
        else:
            extracted_entities[label].add(match_text.lower())
            
        # Return empty string to remove the match from the text
        return ""
    
    # Find matches and replace them with empty string
    modified_resume = re.sub(pattern, replacement_func, modified_resume, flags=re.IGNORECASE)

# Print the extracted entities
print("=== EXTRACTED ENTITIES ===\n")
for label, entities in extracted_entities.items():
    # Convert set to a comma-separated string for printing
    entities_str = ", ".join(sorted(entities))
    print(f"{label}: {entities_str}")

# Print the leftover text
print("\n=== LEFTOVER TEXT ===\n")
print(modified_resume)

=== EXTRACTED ENTITIES ===

Programming Language (PL): c, java, javascript, php, python
Framework (FW): code, d3.js, django, express.js, node.js, react, react.js, spring boot
Database (DB): data structures, database administration, database management, database modelling, database queries, database security, database systems, firebase, graphsql, mongodb, nosql, postgresql, redis, sql
Cloud Platform (CP): aws, aws certified developer, aws with ci, ec2
DevOps (DO): automated testing, ci/cd, docker, git, github, gitlab, jest, load testing, stress testing
Network & Security: oauth, security and
Software Engineering (SWE): bootstrap, front-end, full-stack, jquery, restful apis, user experience, ux, web development
Project Management (PM): agile, scrum
Education Certification (EC): bachelor of science in computer science, certified scrum master, csm, ms

=== LEFTOVER TEXT ===


DAVID CHEN
Software Developer
Boston, MA | david.chen@email.com | (617) 555-1234 | linkedin.com/in/davidchen

SUMMA

In [25]:
# Test job posting
modified_job = job_posting

# Apply regex patterns with wildcard matching
for label, pattern in patterns.items():
    # Define the replacement function inside the loop
    # This creates a new function for each iteration
    def replacement_func(match_obj):
        match_text = match_obj.group(0)
        
        # Add the match to extracted entities
        if isinstance(match_text, tuple):  # Handle tuple case from capture groups
            extracted_entities[label].update(map(str.lower, match_text))
        else:
            extracted_entities[label].add(match_text.lower())
            
        # Return empty string to remove the match from the text
        return ""
    
    # Find matches and replace them with empty string
    modified_text = re.sub(pattern, replacement_func, modified_text, flags=re.IGNORECASE)

# Print the extracted entities
print("=== EXTRACTED ENTITIES ===\n")
for label, entities in extracted_entities.items():
    # Convert set to a comma-separated string for printing
    entities_str = ", ".join(sorted(entities))
    print(f"{label}: {entities_str}")

# Print the leftover text
print("\n=== LEFTOVER TEXT ===\n")
print(modified_job)

=== EXTRACTED ENTITIES ===

Programming Language (PL): c, java, javascript, php, python
Framework (FW): code, d3.js, django, express.js, node.js, react, react.js, spring boot
Database (DB): data structures, database administration, database management, database modelling, database queries, database security, database systems, firebase, graphsql, mongodb, nosql, postgresql, redis, sql
Cloud Platform (CP): aws, aws certified developer, aws with ci, ec2
DevOps (DO): automated testing, ci/cd, docker, git, github, gitlab, jest, load testing, stress testing
Network & Security: oauth, security and
Software Engineering (SWE): bootstrap, front-end, full-stack, jquery, restful apis, user experience, ux, web development
Project Management (PM): agile, scrum
Education Certification (EC): bachelor of science in computer science, certified scrum master, csm, ms

=== LEFTOVER TEXT ===


Senior Software Engineer - Machine Learning
Location: San Francisco, CA (Hybrid)
Salary Range: $150,000 - $180,000 

In [ ]:
# n-grams

import re
import nltk
from nltk.util import ngrams
from collections import defaultdict

#nltk.download('punkt')

# Key words
skill_dictionaries = {
    "Programming Language (PL)" : [
        "python","java", "javascript","c++","c","c#","ruby",'go','php','swift','typescript','kotlin',
        'rust','scala','perl','r','matlab','assembly','visual basic','visual basic for applications'
    ],

    "Framework (FW)": [
        "django", "flask", "spring", "spring framework", "spring boot", "react", "react.js", "reactnative", "angular", "vue", "express", "fastapi", "asp.net", 
        ".net", "laravel", "ruby on rails", "symfony", "meteor", "gatsby", "svelte", "phoenix", "cake", "code"
    ],

    "Database (DB)": [
        "sql", "mysql", "postgresql", "tsql", "plsql", "psql", "nosql", "graphql", "mariadb", "sqlite",
        "mongodb", "couchdb", "influxdb", "documentdb", "dynamodb", "bigtabledb", "cockroachdb", "rethinkdb", "arangodb", "orientdb",
        "datastore", "firestore", "metastore", "blob storage", "object storage", "file storage", "data storage", "disk storage", "cloud storage",
        "database", "knowledgebase", "firebase", "hbase", "database management", "database system", "database administration"
        "data warehouse", "data lake", "data mart", "data repository", "data center", "data server", "data modeling",
        "redis", "cassandra"
    ],

    "Cloud Platform (CP)":[
        "cloud", "cloud computing", "cloud storage", "cloud infrastructure", "cloud platform",
        "aws", "aws lambda", "aws ec2", "aws s3", "aws cloud", "aws elastic beanstalk",
        "aws fargate", "aws sqs", "aws sns", "aws ecs", "aws eks",
        "azure", "azure functions", "azure devops", "azure web apps", "azure kubernetes service", "microsoft azure", 
        "gcp", "google cloud", "google cloud platform", "google cloud functions",
        "amazon web services", "ibm cloud", "oracle cloud", "digital ocean", "heroku",
        "serverless", "iaas", "paas", "saas", "faas", "baas", "caas",
        "s3 bucket", "ec2", "ec2 instance", "lambda", "elastic beanstalk", 
        "ecs", "eks", "fargate", "sqs", "sns", "rds", "redshift"
    ],

    "DevOps (DO)":[
        "devops", "devsecops",
        "ci", "ci/cd", "continuous integration", "continuous delivery", "continuous deployment",
        "continuous integration/continuous delivery", "continuous integration/continuous deployment",
        "version control", "git", "github", "gitlab",       
        "jenkins", "travisci", "circleci", "teamci", "bamboo",    
        "configuration management", "ansible", "puppet", "terraform",   
        "container", "containerization", "containerisation", 
        "docker", "dockerization", "dockerized", "dockerisation", "dockerised", "docker swarm", "kubernetes", "helm", 
        "monitor", "monitoring", "prometheus", "grafana",
        "unit testing", "integration testing", "selenium", "junit", "cypress", "jest",
        "load balancing", "scale", "scaling", "scalability", "disaster recovery", "chaos engineering"           
    ],

    "Network & Security (NS)":[
        "network", "network security", "network engineering", "network architecture",
        "security", "security analyst", "security engineering", "security architecture",
        "routing", "switching", "firewall", "firewall configuration", "vpn", "tcp", "tcp/ip",
        "dns", "dhcp", "ccna", "ccna/ccnp", "sd-wan", "vlans", "subnet",
        "risk", "incident response", "incident management", "identity and access management",
        "iam", "iams", "mfa", "multi-factor authentication", "vulnerability assessment",
        "vulnerability management", "vulnerability analysis",
        "crowdstrike", "carbon black", "cisco", "defense", "sso", "oauth", "saml", "hsm",
        "data encryption", "disk encryption", "owasp zap", "ethical hacking", "penetration testing"
    ],

    "Data Analysis & Science (DAS)":[
        "pandas", "numpy", "scikit-learn", "tensorflow", "power bi", "excel", "tableau", "matplotlib",
        "data analytics", "data analysis", "data science", "data visualisation", "data visualization", "visualizing data",
        "AI", "artificial intelligence", "machine learning", "ML", "NLP", "natural language processing",
        "text analytics", "language model", "language models", "language modeling",
        "transaction management", "data transaction", "data transactions"
    ],

    "Software Engineering (SWE)":[
        "software development", "design patterns", "full stack", "fullstack", "full-stack",
        "software engineer", "software engineering", "junior engineer", "junior software engineer",
        "senior engineer", "senior software engineer", "staff engineer", "principal engineer",
        "code optimization", "code optimisation", "performance tuning", "code refactoring", "refactoring",
        "code review", "code reviews", "peer review", "peer reviews", 'test-driven'
        "microservice", "microservices", "MSA", "microservice architecture",
        "k8s", "eclipse", "eclipse ide", "hibernate", "hibernate orm", "jquery",
        "rest api", "rest apis", "restful api", "restful apis", "restful web service", "restful web services",
        "api development", "web service", "web services",
        "object oriented programming", "oop", "object oriented design", 
        "apache", "apache kafka", "apache tomcat", "apache maven", "apache ant", 
        "apache struts", "apache camel", "apache spark", "apache hadoop", "apache flink",
        "bootstrap", "front-end framework", "rabbitmq", "message queue", "message broker",
        "front-end", "frontend", "back-end", "backend", "web design", "ui", "user interface", "ux", "user experience", "ui/ux",
        "front-end development", "back-end development", "web development"
    ],

    "Project Management (PM)":[
        "agile", 'waterfall', 'atlassian', 'confluence', "jira", "trello", "asana", "kanban", "prince2", "stakeholder management", "scrum"
    ],

    "Education Certification (EC)":[
        "certified scrum master", "csm", "pmp", "aws certified", "azure certified", 
        "gcp certified", "cissp", "ccna", "ceh", "comptia", "cisa", "cism",
        "bachelor", "bachelor of science", "bachelor of engineering", "bachelor of computer science",
        "bachelor of information technology", "bachelor of information systems", 
        "bachelor of cybersecurity", "bachelor of data science", "bachelor of software engineering",
        "B.S.", "BS", "B.E.", "BE", "B.C.S.", "BCS", "B.Tech.", "BTech",
        "master", "master of science", "master of engineering", "master of computer science",
        "master of information technology", "master of information systems",
        "master of cybersecurity", "master of data science", "master of software engineering",
        "M.S.", "MS", "M.E.", "ME", "M.C.S.", "MCS", "M.Tech.", "MTech",
        "phd", "ph.d.", "doctorate", "doctor of philosophy",
        "specialisation in software", "specialisation in data", "specialisation in cloud",
        "specialisation in security", "specialisation in networking", "specialisation in ai",
        "specialisation in machine learning", "specialization in software", "specialization in data",
        "specialization in cloud", "specialization in security", "specialization in networking",
        "specialization in ai", "specialization in machine learning",
        "minor in computer science", "minor in information technology", "minor in data science",
        "minor in software engineering", "major in computer science", "major in information technology",
        "major in data science", "major in software engineering"
    ],

    "Soft Skills (SS)":[
        "communication", "communication skill", "communication skills", 
        "verbal communication", "written communication",
        "leadership", "leadership skill", "leadership skills", 
        "decision making", "decision-making",
        "team building", "team-building",
        "mentoring", "coaching",
        "strategic thinking", "strategic-thinking",
        "teamwork", "collaboration", "team player",
        "interpersonal", "interpersonal skill", "interpersonal skills",
        "problem solving", "problem-solving",
        "critical thinking", "critical-thinking",
        "adaptability", "flexibility", "creativity",
        "self motivated", "self-motivated",
        "detail oriented", "detail-oriented",
        "work ethic",
        "time management", "time-management",
        "organizational", "organizational skill", "organizational skills",
        "multitasking", "prioritization",
        "project coordination",
        "conflict resolution", "conflict-resolution",
        "emotional intelligence",
        "presentation", "presentation skill", "presentation skills",
        "public speaking", "public-speaking",
        "negotiation", "negotiation skill", "negotiation skills",
        "customer service",
        "active listening", "active-listening",
        "stress management", "stress-management",
        "cultural awareness"
    ],
}

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', ' ', text)
    test = re.sub(r'\s+',' ', text).strip()
    return text

def extract_ngrams_from_text(text, max_n=4):
    processed_text = preprocess_text(text)
    tokens = nltk.word_tokenize(processed_text)
    
    all_ngrams = []
    # Generate n-grams of different lengths (from 1 to max_n)
    for n in range(1, max_n+1):
        n_gram_list = list(ngrams(tokens, n))
        all_ngrams.extend([' '.join(gram) for gram in n_gram_list])
    
    return all_ngrams
    
# Match n-grams against skill dictionaries
def match_skills(ngram_list):
    extracted_skills = defaultdict(set)
    
    for category, skills in skill_dictionaries.items():
        for skill in skills:
            # For single-word skills, require exact match
            if ' ' not in skill:
                if skill in ngram_list:
                    extracted_skills[category].add(skill)
            # For multi-word skills, check for exact or partial matches
            else:
                skill_words = skill.split()
                for ngram in ngram_list:
                    # Exact match for the entire skill
                    if skill == ngram:
                        extracted_skills[category].add(skill)
                        break
                    # For longer n-grams, check if they contain the skill
                    elif len(ngram.split()) > len(skill_words) and skill in ngram:
                        # Calculate how embedded the skill is in the n-gram
                        ngram_words = ngram.split()
                        for i in range(len(ngram_words) - len(skill_words) + 1):
                            if ' '.join(ngram_words[i:i+len(skill_words)]) == skill:
                                extracted_skills[category].add(skill)
                                break
    
    return extracted_skills

In [27]:
# Test resume

resume_ngrams = extract_ngrams_from_text(resume)
resume_skills = match_skills(resume_ngrams)



print("=== EXTRACTED SKILLS FROM RESUME ===\n")
for category, skill_list in resume_skills.items():
    print(f"{category}:")
    for skill in skill_list:
        print(skill)
    print()

=== EXTRACTED SKILLS FROM RESUME ===

Programming Language (PL):
javascript
java
python
php
typescript
c

Framework (FW):
react
spring boot
spring
django
express
code

Database (DB):
nosql
database
postgresql
database management
mongodb
sql
redis
firebase

Cloud Platform (CP):
ec2
aws ec2
aws

DevOps (DO):
git
docker
jest
ci
gitlab
github

Network & Security (NS):
security
oauth

Software Engineering (SWE):
code reviews
restful apis
ux
bootstrap
user experience
full stack
jquery
web development

Project Management (PM):
agile
scrum

Education Certification (EC):
certified scrum master
bachelor
csm
master
aws certified
bachelor of science



In [28]:
#Test job posting

job_ngrams = extract_ngrams_from_text(job_posting)
job_skills = match_skills(job_ngrams)

print("=== EXTRACTED SKILLS FROM JOB POSTING ===\n")
for category, skill_list in job_skills.items():
    print(f"{category}:")
    for skill in skill_list:
        print(skill)
    print()

=== EXTRACTED SKILLS FROM JOB POSTING ===

Programming Language (PL):
java
python

Framework (FW):
code

Database (DB):
nosql
mongodb
postgresql
sql

Cloud Platform (CP):
lambda
aws ec2
cloud infrastructure
aws
ec2
cloud

DevOps (DO):
kubernetes
containerization
monitoring
docker
ci

Data Analysis & Science (DAS):
tensorflow
natural language processing
machine learning
artificial intelligence

Software Engineering (SWE):
senior software engineer
code reviews
microservices
design patterns
restful apis
software engineering
software engineer

Project Management (PM):
agile

Education Certification (EC):
master
bachelor

Soft Skills (SS):
problem solving

